In [212]:
import numpy as np
import pandas as pd

df1 = pd.read_excel('../data/Sex & Age by Year.xlsx',sheet_name='Person',header=2)
display(df1)
area_df = df1[["SA2 code","SA2 name","Postcode"]]
area_df.info()

,Year,S/T code,S/T name,GCCSA code,GCCSA name,SA4 code,SA4 name,SA3 code,SA3 name,SA2 code,...,no..9,no..10,no..11,no..12,no..13,no..14,no..15,no..16,no..17,no..18
0,2001,2,Victoria,2RVIC,Rest of Vic.,201,Ballarat,20101,Ballarat,21001,...,435,368,247,233,184,168,151,67,42,5756
1,2001,2,Victoria,2RVIC,Rest of Vic.,201,Ballarat,20101,Ballarat,21002,...,783,743,513,448,371,440,403,323,373,11497
2,2001,2,Victoria,2RVIC,Rest of Vic.,201,Ballarat,20101,Ballarat,21003,...,1271,1238,913,798,729,723,615,442,387,20268
3,2001,2,Victoria,2RVIC,Rest of Vic.,201,Ballarat,20101,Ballarat,21004,...,1272,1144,872,785,800,820,689,429,310,20424
4,2001,2,Victoria,2RVIC,Rest of Vic.,201,Ballarat,20101,Ballarat,21005,...,407,379,248,179,142,124,84,39,24,5320
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9697,2021,2,Victoria,2RVIC,Rest of Vic.,217,Warrnambool and South West,21703,Colac - Corangamite,21476,...,264,303,363,422,404,311,182,103,71,3978
9698,2021,2,Victoria,2RVIC,Rest of Vic.,217,Warrnambool and South West,21704,Warrnambool,21477,...,444,477,555,526,475,339,210,153,113,6989
9699,2021,2,Victoria,2RVIC,Rest of Vic.,217,Warrnambool and South West,21704,Warrnambool,21478,...,618,675,735,731,781,671,420,228,252,9963
9700,2021,2,Victoria,2RVIC,Rest of Vic.,217,Warrnambool and South West,21704,Warrnambool,21479,...,1374,1435,1444,1404,1245,1150,843,543,504,22462


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9702 entries, 0 to 9701
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   SA2 code  9702 non-null   int64  
 1   SA2 name  9702 non-null   object 
 2   Postcode  8400 non-null   float64
dtypes: float64(1), int64(1), object(1)
memory usage: 227.5+ KB


In [214]:
df2 = pd.DataFrame( {"year":df1["Year"], "area_code":df1["SA2 code"], "Total": df1[ df1.columns[-1]  ]} )  #df1[["Year","SA2 code",]]
df2

,year,area_code,Total
0,2001,21001,5756
1,2001,21002,11497
2,2001,21003,20268
3,2001,21004,20424
4,2001,21005,5320
...,...,...,...
9697,2021,21476,3978
9698,2021,21477,6989
9699,2021,21478,9963
9700,2021,21479,22462


In [215]:
df3 = df1[['SA2 code', 'SA2 name', 'Postcode']]

In [216]:
missing = list(df2[df2['Total'] == 0]['area_code'].drop_duplicates())
missing

[21080, 21083, 21127, 21192, 21227]

In [217]:
for area_code in missing:
    df2 = df2[df2['area_code'] != area_code]

In [218]:
from unittest import result
from sklearn.linear_model import LinearRegression
population_list = []
for area_code,sub_df in df2.groupby("area_code"):  
    pct_change_arr = list(pd.Series.pct_change(sub_df['Total']).values[1:])
    X = pd.DataFrame(sub_df[['year']])[1:]
    lr  = LinearRegression()
    lr.fit(X, pct_change_arr)
    predict = lr.predict(X = pd.DataFrame(list(range(2022, 2027)),columns= ['year']))
    population_2021 = list(sub_df['Total'])[-1] 
    population_arr = list(sub_df['Total'])
    year = 2022
    for i in range(5):
        last_population = population_arr[i+20]
        population = round(last_population*(1+predict[i]))
        population_arr.append(population)
        population_list.append([year+i, area_code, population])
  
predict_df = pd.DataFrame(population_list, columns=['year','area_code', 'Total'])
result_df = pd.concat([df2, predict_df])
   



In [219]:
result_df = result_df.join(df3)
result_df.to_csv("../data/population_forcasting.csv")